# 3   Processing Raw Text

In [79]:
import nltk, re, pprint
from nltk import word_tokenize

get .txt from internet ready for processing

In [80]:
from urllib import request

url = "http://www.gutenberg.org/cache/epub/43821/pg43821.txt"
response = request.urlopen(url)
hym_raw = response.read().decode("utf8")
hym_raw[:75]

'\ufeffThe Project Gutenberg EBook of Hymnen an die Nacht / Die Christenheit oder'

then tokenize:

In [81]:
tokens = word_tokenize(hym_raw)

If we now take the further step of creating an NLTK text from this list, we can carry out all of the other linguistic processing we saw in 1., along with the regular list operations like slicing:

In [82]:
text = nltk.Text(tokens)

In [83]:
text.collocations()

Project Gutenberg-tm; Project Gutenberg; Literary Archive; Archive
Foundation; Gutenberg-tm electronic; Gutenberg Literary; electronic
works; United States; set forth; public domain; electronic work;
Gutenberg-tm License; auf Erden; copyright holder; PROJECT GUTENBERG;
Christenheit oder; oder Europa; Die Christenheit; eine neue; unter den


We cannot reliably detect where the content begins and ends, and so have to resort to manual inspection of the file, to discover unique strings that mark the beginning and the end, before trimming raw to be just the content and nothing else:

In [84]:
hym_raw.find("Welcher Lebendige, Sinnbegabte") # first pos

1712

In [85]:
hym_raw.rfind("Die beiden Werke von Novalis")

65130

In [86]:
new_hym_raw = hym_raw[1712:65130]
hym_tokens = word_tokenize(new_hym_raw)
text = nltk.Text(hym_tokens)

In [87]:
bigram = list(nltk.bigrams(text))

def generate_model(cfdist, word, num=10):
    for i in range(num):
        print(word, end =" ")
        word = cfdist[word].max()

cfd = nltk.ConditionalFreqDist(bigram)
generate_model(cfd, "Morgen")

Morgen wiederkommen , und die Geschichte des Lebens zu sein 

# working with htmls

In [149]:
from bs4 import BeautifulSoup

In [158]:
url = "http://news.bbc.co.uk/2/hi/health/2284783.stm"
html = request.urlopen(url).read().decode('utf8')

raw = BeautifulSoup(html, "lxml").get_text()
tokens = word_tokenize(raw)

# etc.

# Encodings

The Python open() function can read encoded data into Unicode strings, and write out Unicode strings in encoded form. It takes a parameter to specify the encoding of the file being read or written. So let's open our Polish file with the encoding 'latin2' and inspect the contents of the file:

f = open(path, encoding='latin2')

The module unicodedata lets us inspect the properties of Unicode characters. 

# 3.4   Regular Expressions for Detecting Word Patterns

Many linguistic processing tasks involve pattern matching. For example, we can find words ending with ed using endswith('ed'). We saw a variety of such "word tests" in 4.2. Regular expressions give us a more powerful and flexible method for describing the character patterns we are interested in.

In [163]:
wordlist = [w for w in nltk.corpus.words.words('en') if w.islower()]

Let's find words ending with ed using the regular expression «ed$». We will use the re.search(p, s) function to check whether the pattern p can be found somewhere inside the string s. We need to specify the characters of interest, and use the **dollar sign** which has a special behavior in the context of regular expressions in that it **matches the end of the word**:

In [168]:
[w for w in wordlist if re.search('ouse$', w)]

['alehouse',
 'almshouse',
 'archhouse',
 'arouse',
 'babyhouse',
 'backhouse',
 'baghouse',
 'bakehouse',
 'bargehouse',
 'bathhouse',
 'bawdyhouse',
 'beadhouse',
 'beamhouse',
 'bedouse',
 'beehouse',
 'beerhouse',
 'bellhouse',
 'bespouse',
 'birdhouse',
 'bleachhouse',
 'blockhouse',
 'blouse',
 'boardinghouse',
 'boathouse',
 'boilerhouse',
 'bouse',
 'brewhouse',
 'brinehouse',
 'bughouse',
 'bunkhouse',
 'cakehouse',
 'carhouse',
 'carouse',
 'casthouse',
 'charterhouse',
 'charthouse',
 'chophouse',
 'chouse',
 'clearinghouse',
 'clockhouse',
 'clubhouse',
 'coalmouse',
 'cockarouse',
 'coffeehouse',
 'colemouse',
 'cookhouse',
 'coolhouse',
 'cornhouse',
 'countinghouse',
 'courthouse',
 'creepmouse',
 'crouse',
 'crucethouse',
 'cubbyhouse',
 'culverhouse',
 'customhouse',
 'deadhouse',
 'deckhouse',
 'delouse',
 'deyhouse',
 'dishouse',
 'dockhouse',
 'doghouse',
 'dollhouse',
 'dormouse',
 'douse',
 'dovehouse',
 'draughthouse',
 'dryhouse',
 'duckhouse',
 'dyehouse',
 'en

The **. wildcard symbol matches any single character**. Suppose we have room in a crossword puzzle for an 8-letter word with j as its third letter and t as its sixth letter. In place of each blank cell we use a period:

The **caret symbol ^ matches the start of a string**, just like the $ matches the end. 

In [169]:
[w for w in wordlist if re.search("^..j..t..$", w)]

['abjectly',
 'adjuster',
 'dejected',
 'dejectly',
 'injector',
 'majestic',
 'objectee',
 'objector',
 'rejecter',
 'rejector',
 'unjilted',
 'unjolted',
 'unjustly']

Finally, **the ? symbol specifies that the previous character is optional**. 
Thus "^e-?mail$" will match both email and e-mail. We could count the total number of occurrences of this word (in either spelling) in a text using  sum(1 for w in text if re.search('^e-?mail$', w)).

The T9 system is used for entering text on mobile phones (see 3.5). Two or more words that are entered with the same sequence of keystrokes are known as textonyms. For example, both hole and golf are entered by pressing the sequence 4653. What other words could be produced with the same sequence? Here we use the regular expression «^[ghi][mno][jlk][def]$»:



In [180]:
[w for w in wordlist if re.search("^[ghi][mno][jlk][def]$", w)]

['gold', 'golf', 'hold', 'hole']

**+ simply means "one or more instances of the preceding item"**, which could be an individual character like m, a set like [fed] or a range like [d-f]. Now let's replace + with *****, which **means "zero or more instances of the preceding item".** 

The regular expression «^m*i*n*e*$» will match everything that we found using «^m+i+n+e+$», but also words where some of the letters don't appear at all, e.g. me, min, and mmmmm. Note that the + and * symbols are sometimes referred to as Kleene closures, or simply closures.

In [181]:
chat_words = sorted(set(w for w in nltk.corpus.nps_chat.words()))

In [182]:
[w for w in chat_words if re.search('^m+i+n+e+$', w)]

['miiiiiiiiiiiiinnnnnnnnnnneeeeeeeeee',
 'miiiiiinnnnnnnnnneeeeeeee',
 'mine',
 'mmmmmmmmiiiiiiiiinnnnnnnnneeeeeeee']

In [183]:
[w for w in chat_words if re.search('^[ha]+$', w)]

['a',
 'aaaaaaaaaaaaaaaaa',
 'aaahhhh',
 'ah',
 'ahah',
 'ahahah',
 'ahh',
 'ahhahahaha',
 'ahhh',
 'ahhhh',
 'ahhhhhh',
 'ahhhhhhhhhhhhhh',
 'h',
 'ha',
 'haaa',
 'hah',
 'haha',
 'hahaaa',
 'hahah',
 'hahaha',
 'hahahaa',
 'hahahah',
 'hahahaha',
 'hahahahaaa',
 'hahahahahaha',
 'hahahahahahaha',
 'hahahahahahahahahahahahahahahaha',
 'hahahhahah',
 'hahhahahaha']

The **^ operator has another function when it appears as the first character inside square brackets.** For example «[^aeiouAEIOU]» **matches any character other than** a vowel. We can search the NPS Chat Corpus for words that are made up entirely of non-vowel characters using «^[^aeiouAEIOU]+$» to find items like these: :):):), grrr, cyb3r and zzzzzzzz. Notice this includes non-alphabetic characters.

In [186]:
[w for w in chat_words if re.search("^[^aeiouAEIOU]+$", w)]

['!',
 '!!',
 '!!!',
 '!!!!',
 '!!!!!',
 '!!!!!!',
 '!!!!!!!',
 '!!!!!!!!',
 '!!!!!!!!!',
 '!!!!!!!!!!',
 '!!!!!!!!!!!',
 '!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!!!!!!!',
 '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!',
 '!!!!!!.',
 '!!!!!.',
 '!!!!....',
 '!!!.',
 '!!.',
 '!!...',
 '!.',
 '!...',
 '!=',
 '!?',
 '!??',
 '!???',
 '"',
 '"...',
 '"?',
 '"s',
 '#',
 '###',
 '####',
 '$',
 '$$',
 '$27',
 '&',
 '&^',
 "'",
 "''",
 "'.",
 "'d",
 "'ll",
 "'m",
 "'n'",
 "'s",
 '(',
 '(((',
 '((((',
 '(((((',
 '((((((',
 '(((((((',
 '((((((((',
 '(((((((((',
 '((((((((((',
 '(((((((((((',
 '((((((((((((',
 '(((((((((((((',
 '((((((((((((((',
 '(((((((((((((((',
 '(((((((((((((((((',
 '((((((((((((((((((',
 '((((((((((((((((((((',
 '(((((((((((((((((((((',
 '(((((((((((((((((((((((',
 '((((((((((((((((((((((((',
 '(((((((((((((((((((((((((',
 '((((((((((((((((((((((((((',
 '((((

**a backslash means that the following character is deprived of its special powers and must literally match a specific character in the word.** Thus, while . is special, \. only matches a period. 

**The braced expressions, like {3,5}, specify the number of repeats of the previous item.**

**The pipe character indicates a choice between the material on its left or its right.**

**Parentheses indicate the scope of an operator: they can be used together with the pipe (or disjunction) symbol** like this: «w(i|e|ai|oo)t», matching wit, wet, wait, and woot. It is instructive to see what happens when you omit the parentheses from the last expression above, and search for «ed|ing$».

In [187]:
wsj = sorted(set(nltk.corpus.treebank.words()))

In [188]:
[w for w in wsj if re.search('^[0-9]+\.[0-9]+$', w)]

['0.0085',
 '0.05',
 '0.1',
 '0.16',
 '0.2',
 '0.25',
 '0.28',
 '0.3',
 '0.4',
 '0.5',
 '0.50',
 '0.54',
 '0.56',
 '0.60',
 '0.7',
 '0.82',
 '0.84',
 '0.9',
 '0.95',
 '0.99',
 '1.01',
 '1.1',
 '1.125',
 '1.14',
 '1.1650',
 '1.17',
 '1.18',
 '1.19',
 '1.2',
 '1.20',
 '1.24',
 '1.25',
 '1.26',
 '1.28',
 '1.35',
 '1.39',
 '1.4',
 '1.457',
 '1.46',
 '1.49',
 '1.5',
 '1.50',
 '1.55',
 '1.56',
 '1.5755',
 '1.5805',
 '1.6',
 '1.61',
 '1.637',
 '1.64',
 '1.65',
 '1.7',
 '1.75',
 '1.76',
 '1.8',
 '1.82',
 '1.8415',
 '1.85',
 '1.8500',
 '1.9',
 '1.916',
 '1.92',
 '10.19',
 '10.2',
 '10.5',
 '107.03',
 '107.9',
 '109.73',
 '11.10',
 '11.5',
 '11.57',
 '11.6',
 '11.72',
 '11.95',
 '112.9',
 '113.2',
 '116.3',
 '116.4',
 '116.7',
 '116.9',
 '118.6',
 '12.09',
 '12.5',
 '12.52',
 '12.68',
 '12.7',
 '12.82',
 '12.97',
 '120.7',
 '1206.26',
 '121.6',
 '126.1',
 '126.15',
 '127.03',
 '129.91',
 '13.1',
 '13.15',
 '13.5',
 '13.50',
 '13.625',
 '13.65',
 '13.73',
 '13.8',
 '13.90',
 '130.6',
 '130.7',
 '

In [189]:
[w for w in wsj if re.search('^[A-Z]+\$$', w)]

['C$', 'US$']

In [190]:
[w for w in wsj if re.search('^[0-9]{4}$', w)]

['1614',
 '1637',
 '1787',
 '1901',
 '1903',
 '1917',
 '1925',
 '1929',
 '1933',
 '1934',
 '1948',
 '1953',
 '1955',
 '1956',
 '1961',
 '1965',
 '1966',
 '1967',
 '1968',
 '1969',
 '1970',
 '1971',
 '1972',
 '1973',
 '1975',
 '1976',
 '1977',
 '1979',
 '1980',
 '1981',
 '1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2005',
 '2009',
 '2017',
 '2019',
 '2029',
 '3057',
 '8300']

In [191]:
[w for w in wsj if re.search('^[0-9]+-[a-z]{3,5}$', w)]

['10-day',
 '10-lap',
 '10-year',
 '100-share',
 '12-point',
 '12-year',
 '14-hour',
 '15-day',
 '150-point',
 '190-point',
 '20-point',
 '20-stock',
 '21-month',
 '237-seat',
 '240-page',
 '27-year',
 '30-day',
 '30-point',
 '30-share',
 '30-year',
 '300-day',
 '36-day',
 '36-store',
 '42-year',
 '50-state',
 '500-stock',
 '52-week',
 '69-point',
 '84-month',
 '87-store',
 '90-day']

In [192]:
[w for w in wsj if re.search('^[a-z]{5,}-[a-z]{2,3}-[a-z]{,6}$', w)]

['black-and-white',
 'bread-and-butter',
 'father-in-law',
 'machine-gun-toting',
 'savings-and-loan']

In [195]:
[w for w in wsj if re.search('(ed|ing)$', w)]

['62%-owned',
 'Absorbed',
 'According',
 'Adopting',
 'Advanced',
 'Advancing',
 'Alfred',
 'Allied',
 'Annualized',
 'Anything',
 'Arbitrage-related',
 'Arbitraging',
 'Asked',
 'Assuming',
 'Atlanta-based',
 'Baking',
 'Banking',
 'Beginning',
 'Beijing',
 'Being',
 'Bermuda-based',
 'Betting',
 'Boeing',
 'Broadcasting',
 'Bucking',
 'Buying',
 'Calif.-based',
 'Change-ringing',
 'Citing',
 'Concerned',
 'Confronted',
 'Conn.based',
 'Consolidated',
 'Continued',
 'Continuing',
 'Declining',
 'Defending',
 'Depending',
 'Designated',
 'Determining',
 'Developed',
 'Died',
 'During',
 'Encouraged',
 'Encouraging',
 'English-speaking',
 'Estimated',
 'Everything',
 'Excluding',
 'Exxon-owned',
 'Faulding',
 'Fed',
 'Feeding',
 'Filling',
 'Filmed',
 'Financing',
 'Following',
 'Founded',
 'Fracturing',
 'Francisco-based',
 'Fred',
 'Funded',
 'Funding',
 'Generalized',
 'Germany-based',
 'Getting',
 'Guaranteed',
 'Having',
 'Heating',
 'Heightened',
 'Holding',
 'Housing',
 'Illumin

.	Wildcard, matches any character
^abc	Matches some pattern abc at the start of a string
abc$	Matches some pattern abc at the end of a string
[abc]	Matches one of a set of characters
[A-Z0-9]	Matches one of a range of characters
ed|ing|s	Matches one of the specified strings (disjunction)
*	Zero or more of previous item, e.g. a*, [a-z]* (also known as Kleene Closure)
+	One or more of previous item, e.g. a+, [a-z]+
?	Zero or one of the previous item (i.e. optional), e.g. a?, [a-z]?
{n}	Exactly n repeats where n is a non-negative integer
{n,}	At least n repeats
{,n}	No more than n repeats
{m,n}	At least m and no more than n repeats
a(b|c)+	Parentheses that indicate the scope of the operators

when using regular expressions containing backslash, we should instruct the interpreter not to look inside the string at all, but simply to pass it directly to the re library for processing. We do this by prefixing the string with the letter r, to indicate that it is a raw string. For example, the raw string r'\band\b' contains two \b symbols that are interpreted by the re library as matching word boundaries instead of backspace characters. If you get into the habit of using r'...' for regular expressions — as we will do from now on — you will avoid having to think about these complications.

# 3.5   Useful Applications of Regular Expressions

The re.findall() ("find all") method finds all (non-overlapping) matches of the given regular expression. Let's find all the vowels in a word, then count them:



In [206]:
[n.split() for n in re.findall(r"[0-9]{4}.[0-9]{1,}.[0-9]{1,}", '2009-12-31')]



[['2009-12-31']]

In [213]:
def stem(word):
    regexp = r'^(.*?)(ing|ly|ed|ious|ies|ive|es|s|ment)?$'
    stem, suffix = re.findall(regexp, word)[0]
    return stem

raw = "Listen, strange women lying in ponds distributing swords is no basis for a system of government.  Supreme executive power derives from a mandate from the masses, not from some farcical"

tok = word_tokenize(raw)

[stem(t) for t in tok]

['Listen',
 ',',
 'strange',
 'women',
 'ly',
 'in',
 'pond',
 'distribut',
 'sword',
 'i',
 'no',
 'basi',
 'for',
 'a',
 'system',
 'of',
 'govern',
 '.',
 'Supreme',
 'execut',
 'power',
 'deriv',
 'from',
 'a',
 'mandate',
 'from',
 'the',
 'mass',
 ',',
 'not',
 'from',
 'some',
 'farcical']

Notice that our regular expression removed the s from ponds but also from is and basis. It produced some non-words like distribut and deriv, but these are acceptable stems in some applications.

You can use a special kind of regular expression for searching across multiple words in a text (where a text is a list of tokens). For example, "<a> <man>" finds all instances of a man in the text. The angle brackets are used to mark token boundaries, and any whitespace between the angle brackets is ignored (behaviors that are unique to NLTK's findall() method for texts). In the following example, we include <.*> [1] which will match any single token, and enclose it in parentheses so only the matched word (e.g. monied) and not the matched phrase (e.g. a monied man) is produced. The second example finds three-word phrases ending with the word bro [2]. The last example finds sequences of three or more words starting with the letter l [3].

In [214]:
from nltk.corpus import gutenberg, nps_chat
moby = nltk.Text(gutenberg.words('melville-moby_dick.txt'))

In [217]:
moby.findall(r"<a> (<.*>) <man>") # () = only this word is printed

monied; nervous; dangerous; white; white; white; pious; queer; good;
mature; white; Cape; great; wise; wise; butterless; white; fiendish;
pale; furious; better; certain; complete; dismasted; younger; brave;
brave; brave; brave
